In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
import os

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
midi_dir = './drive/My Drive/midi_songs'
import glob
from music21 import converter, instrument, note, chord, stream

In [0]:
notes = []
for file in glob.glob(midi_dir + "/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

In [0]:
print(len(notes))

57177
57177


In [0]:
n_vocab = len(set(notes))
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note2idx = dict((note, number) for number, note in enumerate(pitchnames))
idx2note = np.array(pitchnames)
notes_as_int = np.array([note2idx[n] for n in notes])

In [0]:
print('{')
for char, _ in zip(note2idx, range(5)):
    print('  {:4s}: {:3d},'.format(repr(char), note2idx[char]))
print('  ...\n}')

{
  '0' :   0,
  '0.1':   1,
  '0.1.5':   2,
  '0.1.6':   3,
  '0.2':   4,
  ...
}
{
  '0' :   0,
  '0.1':   1,
  '0.1.5':   2,
  '0.1.6':   3,
  '0.2':   4,
  ...
}


In [0]:
print ('{} ---- notes mapped to int ---- > {}'.format(repr(notes[:13]), notes_as_int[:13]))

['4.9', 'E2', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', '11.4', '4.9', '11.4', '4.9'] ---- notes mapped to int ---- > [168 329 168 168 168 168 168 168 168  89 168  89 168]
['4.9', 'E2', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', '4.9', '11.4', '4.9', '11.4', '4.9'] ---- notes mapped to int ---- > [168 329 168 168 168 168 168 168 168  89 168  89 168]


In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 150
examples_per_epoch = len(notes)//(seq_length+1)

# Create training examples / targets
notes_dataset = tf.data.Dataset.from_tensor_slices(notes_as_int)

for i in notes_dataset.take(5):
  print(idx2note[i.numpy()])

4.9
E2
4.9
4.9
4.9
4.9
E2
4.9
4.9
4.9


In [0]:
sequences = notes_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2note[item.numpy()])))

'4.9E24.94.94.94.94.94.94.911.44.911.44.94.94.94.94.90.4E24.90.44.94.94.94.94.99.24.99.29.24.94.94.94.94.94.9E24.94.94.94.94.9E5F5G#5A54.94.95.114.95.114.94.94.9E5F5G#5A54.94.99.0E24.99.04.94.94.9E5F5G#5A54.94.911.24.911.211.24.94.94.9E5F5G#5A54.94.93.7.11E-23.7.11B2G21.5.9F#21.5.93.7.11E-23.7.11G2B2E-31.5.9G#21.5.91.5.9F3F2F2F2F2F24.9E54.9C54.9A54.95.9F55.9C55.9A55.94.9E54.9C54.9A54.9F55.9C55.9E55.9D55.9E54.9E-54.9B54.94.9A55.95.95.95.9'
'4.94.94.94.95.95.95.95.9B44.9A44.9E54.94.9E-55.95.95.95.94.94.94.94.95.95.95.95.9E54E-5C6E55E-5B5E56E-5C6A55A44C5E5F5E55C5A4C5A46C5E5F#5E55C5A4C5A44C5E5F5E55C5A4C5A46C5E5F#5E55C5A4C5A4A2C5E5F5A2E5A2C5B4C5A4A2F4E4F4A2E4A2C4B3C4E54.9B54.9A54.9G5F54.94.9E55.10F5G5D55.10F55.10E55.10D55.10E54.9B4C54.9D54.9C54.9B44.9C50.5B4C5A40.5F40.5G40.54.94.94.94.9'
'4.9G54.9F#5E54.94.99.114.99.114.94.9E-5D54.9C5D54.9E54.99.04.99.04.94.9C64.9G#5A54.9C64.911.24.911.211.24.94.9C64.9G5G#5B5A54.9E54.9B54.9A54.9G5F54.94.9E55.10F5G5D55.10F55.10E55.10D55.10E54.9B4C54.9D54.9C5

In [0]:
def split_input_target(chunk):
    input_notes = chunk[:-1]
    target_notes = chunk[1:]
    return input_notes, target_notes

dataset = sequences.map(split_input_target)

In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2note[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2note[target_example.numpy()])))

Input data:  '4.9E24.94.94.94.94.94.94.911.44.911.44.94.94.94.94.90.4E24.90.44.94.94.94.94.99.24.99.29.24.94.94.94.94.94.9E24.94.94.94.94.9E5F5G#5A54.94.95.114.95.114.94.94.9E5F5G#5A54.94.99.0E24.99.04.94.94.9E5F5G#5A54.94.911.24.911.211.24.94.94.9E5F5G#5A54.94.93.7.11E-23.7.11B2G21.5.9F#21.5.93.7.11E-23.7.11G2B2E-31.5.9G#21.5.91.5.9F3F2F2F2F2F24.9E54.9C54.9A54.95.9F55.9C55.9A55.94.9E54.9C54.9A54.9F55.9C55.9E55.9D55.9E54.9E-54.9B54.94.9A55.95.95.9'
Target data: 'E24.94.94.94.94.94.94.911.44.911.44.94.94.94.94.90.4E24.90.44.94.94.94.94.99.24.99.29.24.94.94.94.94.94.9E24.94.94.94.94.9E5F5G#5A54.94.95.114.95.114.94.94.9E5F5G#5A54.94.99.0E24.99.04.94.94.9E5F5G#5A54.94.911.24.911.211.24.94.94.9E5F5G#5A54.94.93.7.11E-23.7.11B2G21.5.9F#21.5.93.7.11E-23.7.11G2B2E-31.5.9G#21.5.91.5.9F3F2F2F2F2F24.9E54.9C54.9A54.95.9F55.9C55.9A55.94.9E54.9C54.9A54.9F55.9C55.9E55.9D55.9E54.9E-54.9B54.94.9A55.95.95.95.9'
Input data:  '4.9E24.94.94.94.94.94.94.911.44.911.44.94.94.94.94.90.4E24.90.44.94.94.94.94.99.

In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2note[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2note[target_idx])))

Step    0
  input: 168 ('4.9')
  expected output: 329 ('E2')
Step    1
  input: 329 ('E2')
  expected output: 168 ('4.9')
Step    2
  input: 168 ('4.9')
  expected output: 168 ('4.9')
Step    3
  input: 168 ('4.9')
  expected output: 168 ('4.9')
Step    4
  input: 168 ('4.9')
  expected output: 168 ('4.9')
Step    0
  input: 168 ('4.9')
  expected output: 329 ('E2')
Step    1
  input: 329 ('E2')
  expected output: 168 ('4.9')
Step    2
  input: 168 ('4.9')
  expected output: 168 ('4.9')
Step    3
  input: 168 ('4.9')
  expected output: 168 ('4.9')
Step    4
  input: 168 ('4.9')
  expected output: 168 ('4.9')


In [0]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 1000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 150), (64, 150)), types: (tf.int64, tf.int64)>

<BatchDataset shapes: ((64, 150), (64, 150)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(pitchnames)

# The embedding dimension
embedding_dim = 64

# Number of RNN units
rnn_units = 128

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(0.15),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)


In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  print(input_example_batch.numpy().shape)
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 150)
(64, 150, 358) # (batch_size, sequence_length, vocab_size)
(64, 150)
(64, 150, 358) # (batch_size, sequence_length, vocab_size)


In [0]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (64, None, 64)            22912     
_________________________________________________________________
lstm_4 (LSTM)                (64, None, 128)           98816     
_________________________________________________________________
dropout_6 (Dropout)          (64, None, 128)           0         
_________________________________________________________________
dense_6 (Dense)              (64, None, 358)           46182     
Total params: 167,910
Trainable params: 167,910
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (64, None, 64)            22912     
__________________________

In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([232, 290, 279,  52, 318, 235, 313,  62, 133, 174, 296,  93, 125,
       354, 303, 248, 194, 191, 290,  32,  76, 197, 272,  13, 146, 224,
       270, 223, 341,  11, 162, 183,  65, 256, 215, 119, 157, 341,  76,
       144,  88, 276,  54, 352, 153,   8, 200, 200, 304,  27, 116, 307,
       325,   5, 196, 206, 254, 347, 104,  20,  86,  70,  15, 327,  95,
        87,  51, 314, 245, 337,  83, 339, 245, 299, 329, 339, 248, 341,
        75, 123,  32, 329, 137, 155, 123, 148, 176, 325, 219, 272, 143,
       205, 143, 255, 222,  96,   4,  93,  55,  63, 168, 339, 338, 222,
       140,  25, 248, 282,  39, 125, 146, 310, 226, 160,  47, 176, 290,
       167,   9, 229, 184,  83, 291, 352,  43,   4,  69,  48,  66,   8,
       296, 248, 126, 241, 311, 151, 178, 310, 208, 223,  73, 172,  37,
       291, 310, 202, 281, 234,  36, 118])

array([  1, 351, 141, 172, 352,  57, 147, 320, 144,  76, 165, 264,  49,
       270, 309, 177,  65, 310,  27,  12, 103, 169,  45, 328, 136, 161,
        25,  74, 287, 260, 180, 153, 207, 296,  23,  35, 330,  13, 218,
       155, 266, 149,  42, 249,  56, 266, 112, 241, 338, 206, 126, 326,
        40, 340, 144, 219,  97,  89, 167,  96,  69, 281,   5,  34,  78,
       160, 319, 168, 296, 172,  89,  16,  31, 259, 173, 158,  59,  19,
       134, 247, 334,  64, 356, 329,  32, 190, 168, 184, 142, 288, 241,
       190, 280,  29,  35,  92, 161, 113, 122, 116, 266, 120, 125, 209,
       281, 330, 187, 298, 170, 265, 301, 250, 127, 332, 120, 289, 313,
        63, 138, 108, 274, 274,  38,  88, 272,  98,  62, 231, 151, 224,
       342, 149, 334, 299,  34, 209, 142, 321,  63, 272, 113, 102, 220,
       270, 248, 193, 313, 178,  60, 173])

In [0]:
print("Input: \n", repr("".join(idx2note[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2note[sampled_indices ])))
sampled_indices.shape

Input: 
 'F#4D4C#4F#37.11E5E2F#5B2D5E3F#3G3F#4G4B4E4D5D4A2D47.11E5E3G3B3D4B3C#47.9A4D5D3A3E5C#4A3F#5D4A3A5F#4C#6A3F#2E34.9E5C#4D5C#5A3E4D5A5F#3A32.7C#6G2D6D3B5G3A3B3A5D4B5G4A5A2E3A37.11.2E311.1.4A3A52.6B2F#3C#42.6F#31.6E4F#36.11D46.11F#3F#24.9E3B3E3C#4B31.6F#59.2A511.2B5G2D3A3B5G3A5B3G56.11D5G#3F#5A2E5E3A3D4C#4G2E4A4C#5D5F#2D3E5A3E4F#5D4G5A5E2B2E3F#3G3D4B3A5G5D5F#511.26.7'

Next Char Predictions: 
 '7.9.11.2B-39.210.0.5D48.0C610.11.3.53.6.85.6B22.3.73.5.7G3C#38.11.16.11.06.10.11B-31.3.611.1.4.56.7.119.11.0.40.3.64.5.7.117.8.09.10.2.47.8F20.3.54.7.9.115.910.2.48.9.11.17.10.034.7F211.1.4.54.1011.3.69.11.2.510.1.3G14.6.70.2.66.8.06.8.0C#41.2.62.7C#7E-40.2.3.76.7.06.98.9.1.3G#22.50.4.711.311.0.40.3.6.9E-62.3.7.911.3.510.0.4C78.10.2F#411.2.5F#68.10.2B5E2F#68.11.1F211.1.43.51.3.6E23.7.114.6.93.54.5.9.05.7.0E-47.10.29.11.0.446.8.11.248.9.1.47.11.22.40.22.3.710.1.3.5.610.24.9F#6F#57.11.23.81.28.11.1A11.4.7.103.5.74.5.7.11C37.8.10.24.7.9105.7.0B-34.8.9.110.2.77.9.05.9.011.2.5B-4G11.5.80.211.010

(150,)

Input: 
 'A3D59.0E5A3E4C4B3F#5G5D4F#5E5G3F#3E5E3E-5E-3E54.7.11A4A3A3B4C5C4E4A3D59.0E5A3E4B3C4F#5G5D4F#52.6.9E5D5A3E5B3C54.9B47.11A49.0E44.7C5A3A3C5D5B3C5C4E4B4A3A49.0G4E3A3E5C4B4B3D5D4C5B4G3F#3A4E3G#4E-3A49.0.4A4A3A3B4C5C4E4A3D59.0E5A3E4B3C4F#5G5D4F#52.6.9E5D5A3E5B3C54.9B47.11A49.0E44.7A4A3A3B4B3C5C4E4A3D59.0E5A3E4C4B3F#5G5D4F#5E5G3F#3E5E3E-5E-3E54.7.11A5A2'

Next Char Predictions: 
 '0.1G#63.8.95.10.11G110.1.44.5.9D64.1011.1.4.54.8.119.0.3.510.0.2.59.10.2.4C25.7.1010.2.4C31.2.60.3.5.82.4.94.9.101.6E13.7.104.7.9.01.211.1A69.0.25.8.04.6.76.9.0B20.61.4E30.3.67.10.1.34.6.99.14.5.9.111.58.11.1.410.1.3.69.12.6.108.10.0F#56.93.5.7.10E-51.4.7.9F14.107.10.22.4.511.44.8.9.112.411.0A00.2.3.71.3.811.1.64.7.9D54.9B25.10.1111.40.3.71.3.5.89.05.114.7.1010.1.50.4.63.6.9.118.11F#110.2.3G5E21.3.66.10.14.95.9.03.9B-18.10.06.10.19.2.31.31.42.34.7.9.02.6.73.4.8.102.79.13.43.5.76.9.1A0E36B459.0.4C#18.11.23.5.7.8.11E53.4B-2C610.23.7.92.5.8.109.11.29.11.21.4.711.3.69.11.0.42.4.5.910.11.3.57.9.114.6.107.8.0F3

(150,)

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 150, 358)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.880353
Prediction shape:  (64, 150, 358)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.8808246


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './drive/My Drive/training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=500
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/500
5/5 [==============================] - 3s 530ms/step - loss: 5.8734
Epoch 2/500
5/5 [==============================] - 1s 101ms/step - loss: 5.8101
Epoch 3/500
5/5 [==============================] - 0s 93ms/step - loss: 5.3518
Epoch 4/500
5/5 [==============================] - 0s 97ms/step - loss: 4.9948
Epoch 5/500
5/5 [==============================] - 0s 91ms/step - loss: 4.8527
Epoch 6/500
5/5 [==============================] - 0s 99ms/step - loss: 4.8098
Epoch 7/500
5/5 [==============================] - 1s 102ms/step - loss: 4.7360
Epoch 8/500
5/5 [==============================] - 0s 91ms/step - loss: 4.7409
Epoch 9/500
5/5 [==============================] - 0s 97ms/step - loss: 4.7309
Epoch 10/500
5/5 [==============================] - 0s 95ms/step - loss: 4.7215
Epoch 11/500
5/5 [==============================] - 0s 95ms/step - loss: 4.7364
Epoch 12/500
5/5 [==============================] - 0s 93ms/step - loss: 4.7161
Epoch 13/500
5/5 [============================

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

'./drive/My Drive/training_checkpoints/ckpt_500'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (1, None, 64)             22912     
_________________________________________________________________
lstm_7 (LSTM)                (1, None, 128)            98816     
_________________________________________________________________
dropout_9 (Dropout)          (1, None, 128)            0         
_________________________________________________________________
dense_9 (Dense)              (1, None, 358)            46182     
Total params: 167,910
Trainable params: 167,910
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_notes(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = start_string

  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2note[predicted_id])

  return text_generated

In [0]:
start = np.random.randint(low=0, high=vocab_size, size=(20,))
print(start)

[220 214 171 229 290 184 244   6 268 284 168 139  18 280 355 316  49 231
   7 339]


In [0]:
generated_notes = generate_notes(model, start)
print(len(generated_notes))

1000


In [0]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in generated_notes:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [0]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='./drive/My Drive/LSTM_high_loss_song2.mid')

'./drive/My Drive/LSTM_high_loss_song2.mid'